IMPORT LIBRARY

In [1]:
# Import Library
import joblib
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

DATASET

In [2]:
# Muat dataset dari file CSV
data = pd.read_csv('data.csv')
data

,sequence,length,class
0,AATAATTTGTGCACTTCAGAATATTCTACAATAATATATTATTTCC...,1010,DMT2
1,AGTCCTTCGCCGTCCCTCGCCGTCCTTCGCCATCGCACGCCACCGC...,1010,DMT2
2,AGTCCTTCGCCGTCCCTCGCCGTCCTTCGCCATCGCACGCCACCGC...,1013,DMT2
3,AGTTGGAGTCTCCAGGGATCAGGGTTCCAGGAACTCAGGATCTGCA...,1013,DMT2
4,AGTCCTTCGCCGTCCCTCGCCGTCCTTCGCCATCGCACGCCACCGC...,1015,DMT2
...,...,...,...
1572,ATGTGTCGCCTTTTTTTGTTTGCAGTGGAAACAATTTCTCGCCGCT...,8021,DMT1
1573,AGTAGCAGAAAGTGAGGCTGGCAGGCGGCGGCAAAGGAGCCGGCGC...,8088,DMT2
1574,GGCGGCCAGGGAAGGTCTCTGCCGCCAGGGAAGTGTCCCAGAGACC...,8416,DMT1
1575,GCTTAGTGCTGAGCACATCCAGTGGGTAAAGTTCCTTAAAATGCTC...,9032,DMT2


In [3]:
data

,sequence,length,class
0,AATAATTTGTGCACTTCAGAATATTCTACAATAATATATTATTTCC...,1010,DMT2
1,AGTCCTTCGCCGTCCCTCGCCGTCCTTCGCCATCGCACGCCACCGC...,1010,DMT2
2,AGTCCTTCGCCGTCCCTCGCCGTCCTTCGCCATCGCACGCCACCGC...,1013,DMT2
3,AGTTGGAGTCTCCAGGGATCAGGGTTCCAGGAACTCAGGATCTGCA...,1013,DMT2
4,AGTCCTTCGCCGTCCCTCGCCGTCCTTCGCCATCGCACGCCACCGC...,1015,DMT2
...,...,...,...
1572,ATGTGTCGCCTTTTTTTGTTTGCAGTGGAAACAATTTCTCGCCGCT...,8021,DMT1
1573,AGTAGCAGAAAGTGAGGCTGGCAGGCGGCGGCAAAGGAGCCGGCGC...,8088,DMT2
1574,GGCGGCCAGGGAAGGTCTCTGCCGCCAGGGAAGTGTCCCAGAGACC...,8416,DMT1
1575,GCTTAGTGCTGAGCACATCCAGTGGGTAAAGTTCCTTAAAATGCTC...,9032,DMT2


CEK OUTLIER DATA

In [4]:
# Memeriksa apakah ada karakter selain A, C, G, T dalam kolom DNA_sequence
invalid_dna = data[~data['sequence'].str.upper().str.contains('^[ACGT]+$')]

# Tampilkan baris-baris dengan nilai yang tidak valid
invalid_dna

,sequence,length,class


In [5]:
duplikat = data[data.duplicated()]
duplikat

,sequence,length,class


In [6]:
# Jumlah Outlier Data
jumlah_invalid_dna = len(invalid_dna)
print(jumlah_invalid_dna)

0


K-MERS

In [7]:
# Buat fungsi untuk mengonversi sequence DNA menjadi K-mer
def get_kmers(sequence, size=3):
    """Menghasilkan K-mers dari sequence DNA."""
    return [sequence[i:i+size].upper() for i in range(len(sequence) - size + 1)]

# Terapkan K-mer ke kolom 'sequence'
data['kmers'] = data['sequence'].apply(lambda x: get_kmers(x))
data['kmers']

,kmers
0,"[AAT, ATA, TAA, AAT, ATT, TTT, TTG, TGT, GTG, ..."
1,"[AGT, GTC, TCC, CCT, CTT, TTC, TCG, CGC, GCC, ..."
2,"[AGT, GTC, TCC, CCT, CTT, TTC, TCG, CGC, GCC, ..."
3,"[AGT, GTT, TTG, TGG, GGA, GAG, AGT, GTC, TCT, ..."
4,"[AGT, GTC, TCC, CCT, CTT, TTC, TCG, CGC, GCC, ..."
...,...
1572,"[ATG, TGT, GTG, TGT, GTC, TCG, CGC, GCC, CCT, ..."
1573,"[AGT, GTA, TAG, AGC, GCA, CAG, AGA, GAA, AAA, ..."
1574,"[GGC, GCG, CGG, GGC, GCC, CCA, CAG, AGG, GGG, ..."
1575,"[GCT, CTT, TTA, TAG, AGT, GTG, TGC, GCT, CTG, ..."


In [8]:
# Gabungkan semua K-mers menjadi satu string per baris
data['kmers_joined'] = data['kmers'].apply(lambda x: ' '.join(x))
data['kmers_joined']

,kmers_joined
0,AAT ATA TAA AAT ATT TTT TTG TGT GTG TGC GCA CA...
1,AGT GTC TCC CCT CTT TTC TCG CGC GCC CCG CGT GT...
2,AGT GTC TCC CCT CTT TTC TCG CGC GCC CCG CGT GT...
3,AGT GTT TTG TGG GGA GAG AGT GTC TCT CTC TCC CC...
4,AGT GTC TCC CCT CTT TTC TCG CGC GCC CCG CGT GT...
...,...
1572,ATG TGT GTG TGT GTC TCG CGC GCC CCT CTT TTT TT...
1573,AGT GTA TAG AGC GCA CAG AGA GAA AAA AAG AGT GT...
1574,GGC GCG CGG GGC GCC CCA CAG AGG GGG GGA GAA AA...
1575,GCT CTT TTA TAG AGT GTG TGC GCT CTG TGA GAG AG...


Vectorizer

In [9]:
# Use Count Vectorizer for K-mers results
vectorizer = CountVectorizer(ngram_range=(1, 1))
x_kmers = vectorizer.fit_transform(data['kmers_joined'])

# Convert the sparse matrix to DataFrame with feature names as columns
x_kmers_df = pd.DataFrame(x_kmers.toarray(), columns=vectorizer.get_feature_names_out())
x_kmers_df

,aaa,aac,aag,aat,aca,acc,acg,act,aga,agc,...,tcg,tct,tga,tgc,tgg,tgt,tta,ttc,ttg,ttt
0,40,19,27,28,24,5,4,14,24,15,...,3,21,27,14,6,19,20,18,17,30
1,2,2,13,11,9,21,6,9,22,17,...,12,26,20,24,25,27,5,15,12,16
2,6,3,17,10,11,14,6,9,24,14,...,14,22,19,21,24,28,5,15,14,18
3,15,13,23,16,25,12,2,11,36,18,...,0,14,17,18,23,23,10,15,16,17
4,6,3,17,10,11,14,6,9,25,14,...,14,22,19,21,24,28,5,15,14,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1572,288,119,164,145,146,115,52,113,161,128,...,36,164,150,108,126,182,130,170,155,345
1573,256,89,179,112,145,110,38,100,194,154,...,49,178,145,151,169,135,72,147,128,183
1574,45,44,88,19,112,153,40,65,123,191,...,29,124,116,194,309,129,12,66,66,35
1575,385,134,235,225,201,113,20,169,250,120,...,28,193,201,126,200,110,148,180,148,218


In [10]:
# Pisahkan fitur dan label (kolom 'class')
y = data['class']
y.value_counts()

,count
class,
DMG,989
DMT2,443
DMT1,145


OVERSAMPLING

In [11]:
# Terapkan SMOTE untuk Oversampling
ros = SMOTE(random_state=42)
x_resampled, y_smote = ros.fit_resample(x_kmers_df, y)
y_smote.value_counts()

,count
class,
DMT2,989
DMT1,989
DMG,989


Scaler

In [12]:
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x_resampled)

In [13]:
# Create a DataFrame from the scaled NumPy array, using original column names
x_scaled_df = pd.DataFrame(x_scaled, columns=x_kmers_df.columns)

# Display the DataFrame
x_scaled_df

,aaa,aac,aag,aat,aca,acc,acg,act,aga,agc,...,tcg,tct,tga,tgc,tgg,tgt,tta,ttc,ttg,ttt
0,0.099217,0.126866,0.083700,0.104418,0.087629,0.012579,0.045977,0.043210,0.077551,0.027624,...,0.048387,0.091346,0.084211,0.042553,0.013029,0.090047,0.102041,0.060440,0.066351,0.073171
1,0.000000,0.000000,0.022026,0.036145,0.010309,0.113208,0.068966,0.012346,0.069388,0.038674,...,0.193548,0.115385,0.047368,0.095745,0.074919,0.127962,0.025510,0.043956,0.042654,0.035230
2,0.010444,0.007463,0.039648,0.032129,0.020619,0.069182,0.068966,0.012346,0.077551,0.022099,...,0.225806,0.096154,0.042105,0.079787,0.071661,0.132701,0.025510,0.043956,0.052133,0.040650
3,0.033943,0.082090,0.066079,0.056225,0.092784,0.056604,0.022989,0.024691,0.126531,0.044199,...,0.000000,0.057692,0.031579,0.063830,0.068404,0.109005,0.051020,0.043956,0.061611,0.037940
4,0.010444,0.007463,0.039648,0.032129,0.020619,0.069182,0.068966,0.012346,0.081633,0.022099,...,0.225806,0.096154,0.042105,0.079787,0.071661,0.132701,0.025510,0.043956,0.052133,0.040650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,0.039164,0.141791,0.052863,0.044177,0.118557,0.163522,0.103448,0.074074,0.097959,0.160221,...,0.145161,0.192308,0.152632,0.159574,0.237785,0.156398,0.030612,0.148352,0.113744,0.051491
2963,0.151436,0.380597,0.290749,0.168675,0.335052,0.578616,0.586207,0.265432,0.424490,0.541436,...,0.661290,0.278846,0.452632,0.457447,0.322476,0.260664,0.163265,0.340659,0.199052,0.149051
2964,0.075718,0.373134,0.312775,0.108434,0.391753,0.628931,0.448276,0.339506,0.383673,0.591160,...,0.870968,0.447115,0.352632,0.632979,0.426710,0.331754,0.061224,0.445055,0.227488,0.089431
2965,0.002611,0.000000,0.026432,0.036145,0.010309,0.100629,0.068966,0.006173,0.069388,0.038674,...,0.177419,0.110577,0.047368,0.095745,0.074919,0.137441,0.025510,0.043956,0.042654,0.035230


LABEL ENCODER

In [14]:
# Pastikan y adalah array 1D
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y_smote.values.ravel())  # Ubah DataFrame ke array numpy dan ratakan

In [15]:
# Buat DataFrame dengan kelas dan numeriknya
class_mapping = pd.DataFrame({
    'Class': encoder.classes_,
    'Numeric': encoder.transform(encoder.classes_)
})

# Tampilkan DataFrame dalam bentuk tabel
class_mapping

,Class,Numeric
0,DMG,0
1,DMT1,1
2,DMT2,2


In [16]:
# Split data
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_encoded, random_state=42, stratify=y_encoded)

MODELING

In [17]:
# Inisialisasi dan latih model KNN
model = KNeighborsClassifier()
model.fit(x_train, y_train)

# Prediksi pada data uji
y_pred = model.predict(x_test)

EVALUASI

In [18]:
# Evaluasi model
print("Laporan Klasifikasi:\n", classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Laporan Klasifikasi:
               precision    recall  f1-score   support

           0       0.94      0.81      0.87       247
           1       0.81      0.97      0.88       247
           2       0.86      0.82      0.84       248

    accuracy                           0.86       742
   macro avg       0.87      0.86      0.86       742
weighted avg       0.87      0.86      0.86       742

[[199  23  25]
 [  1 239   7]
 [ 12  33 203]]


In [19]:
# Simpan model ke file model.pkl
joblib.dump(model, 'model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(encoder, 'encoder.pkl')

['encoder.pkl']